In [1]:
import csv
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm
import pandas as pd

In [2]:
from urllib.parse import urlparse

# Функция для получения имени файла из URL
def get_filename_from_url(url):
    parsed_url = urlparse(url)
    return os.path.basename(parsed_url.path)

In [3]:
df = pd.read_csv('flats_unique.csv')
flat_urls = df['link']

In [52]:
START_ID = 1901
END_ID = 1970

dir_images = 'images'

err_name = 'download_errors_' + str(START_ID) + '_' + str(END_ID) + '.csv'
csv_errors = open(err_name, mode='w', newline='', encoding='utf-8')
writer_err = csv.writer(csv_errors)
writer_err.writerow(['flat_url', 'image_url', 'image_name'])

FILE_OUT_NAME = 'url_images_' + str(START_ID) + '_' + str(END_ID) + '.csv'
csv_file = open(FILE_OUT_NAME, mode='w', newline='', encoding='utf-8')
writer = csv.writer(csv_file)

writer.writerow(['flat_url', 'image_url', 'image_name'])

31

In [53]:
%%time

for url_id in tqdm(range(START_ID, END_ID+1), desc='Flat images parsing'):
    flat_link = flat_urls[url_id]
    flat_req = requests.get(flat_link)
    flat_soup = BeautifulSoup(flat_req.text, "html.parser")

    # Находим div с классом "fotorama"
    fotorama_div = flat_soup.find('div', class_='fotorama')

    # Ищем все ссылки на фото внутри этого блока
    if fotorama_div is None:
        writer_err.writerow([flat_link, None, None])
    else:
        image_tags = fotorama_div.find_all('a', href=True)
        if image_tags is None:
            writer_err.writerow([flat_link, None, None])
        else:
            # Скачиваем все изображения и записываем данные в CSV
            for i, tag in enumerate(image_tags):
                image_url = tag['href']
                response = requests.get(image_url)

                # Проверяем успешность запроса
                if response.status_code == 200:
                    # Извлекаем оригинальное имя файла из URL
                    filename = get_filename_from_url(image_url)
                    filepath = os.path.join(dir_images, filename)

                    with open(filepath, 'wb') as f:
                        f.write(response.content)

                    # Записываем данные в CSV файл
                    writer.writerow([flat_link, image_url, filename])
                    print(f'Фото {i+1} скачано как {filename} и записано в CSV.')
                else:
                    writer_err.writerow([flat_link, image_url, filename])
                    print(f'Ошибка при скачивании фото {i+1}.')

            print(f'--------end of flat {url_id} / {END_ID} ---------')


csv_file.close()
csv_errors.close()

print('End of program')

Flat images parsing:   1%|▉                                                             | 1/70 [00:04<04:55,  4.29s/it]

Фото 1 скачано как 67ee4159d3e3ce456e436809eedc949c_1200x900.jpg и записано в CSV.
--------end of flat 1901 / 1970 ---------
Фото 1 скачано как a1bff18b9d6aa9ad662a939100daaf8a_1200x900.jpg и записано в CSV.
Фото 2 скачано как fe54802e9087318619f93eb749f074c0_1200x900.jpg и записано в CSV.
Фото 3 скачано как c7b35b5c253dd738de371a418a299f06_1200x900.jpg и записано в CSV.
Фото 4 скачано как 81fe3d7501f7ccacbde6373b90c5d2d2_1200x900.jpg и записано в CSV.
Фото 5 скачано как 9b260513b7dd429a11f24fdf3e2701e2_1200x900.jpg и записано в CSV.


Flat images parsing:   3%|█▊                                                            | 2/70 [00:17<10:59,  9.70s/it]

Фото 6 скачано как 8b7b3c4c0b323ef2dd9d62b35a8e07cd_1200x900.jpg и записано в CSV.
--------end of flat 1902 / 1970 ---------
Фото 1 скачано как 400655771f0e4559d53d1a6e1d709ac3_1200x900.jpg и записано в CSV.
Фото 2 скачано как e5fe76235450234e23cf96f5971c8c11_1200x900.jpg и записано в CSV.
Фото 3 скачано как f83ca6ac7509511c97f2f3001a35538e_1200x900.jpg и записано в CSV.
Фото 4 скачано как 1b19f069e3eb6ba6dcd157ff07089b94_1200x900.jpg и записано в CSV.


Flat images parsing:   4%|██▋                                                           | 3/70 [00:27<10:43,  9.61s/it]

...

Фото 6 скачано как z7rvWKIV-as и записано в CSV.
--------end of flat 1968 / 1970 ---------
Фото 1 скачано как 0e9769d80734f0958384e3eed6e0c550_1200x900.jpg и записано в CSV.
Фото 2 скачано как 0dcefc7f3851e723cd389d794edddf99_1200x900.jpg и записано в CSV.
Фото 3 скачано как a183a648b43234cbdaf768e3302cbadf_1200x900.jpg и записано в CSV.
Фото 4 скачано как ecacde130f586bb3214ccefbeb97b308_1200x900.jpg и записано в CSV.


Flat images parsing:  99%|████████████████████████████████████████████████████████████▏| 69/70 [23:00<00:11, 11.04s/it]

Фото 5 скачано как 479171cf837e3235097cb56e9c02604c_1200x900.jpg и записано в CSV.
--------end of flat 1969 / 1970 ---------
Фото 1 скачано как 1795d0b12a0e446abbad9a10803fafb1_1200x900.jpg и записано в CSV.
Фото 2 скачано как 797a5bb55ac749d0d5f4cc39313e19d5_1200x900.jpg и записано в CSV.
Фото 3 скачано как 7b49a3102b0905f1640936cbc05b5b3c_1200x900.jpg и записано в CSV.
Фото 4 скачано как eb0938b1b5f7bdfc22b26c2fbc325a72_1200x900.jpg и записано в CSV.
Фото 5 скачано как 75ae4f02ef1c375ac25a0c1843c70bde_1200x900.jpg и записано в CSV.
Фото 6 скачано как 7b9b129f4b7d769f3c20b8cef24fc155_1200x900.jpg и записано в CSV.


Flat images parsing: 100%|█████████████████████████████████████████████████████████████| 70/70 [23:12<00:00, 19.90s/it]

Фото 7 скачано как 3AQXYx и записано в CSV.
--------end of flat 1970 / 1970 ---------
End of program
CPU times: total: 8min 40s
Wall time: 23min 13s


In [29]:
fotorama_div